In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

2023-10-25 18:12:08.501901: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-25 18:12:08.503504: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-25 18:12:08.535466: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-25 18:12:08.535978: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-25 18:12:09.015296: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
data = pd.read_csv("../data.csv")
data.head()

,Unnamed: 0.1,Unnamed: 0,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
0,0,2010-01-04,0.9133,1.4419,1.7200,1.0377,6.8273,5.1597,7.7555,46.27,92.55,3.396
1,1,2010-01-05,0.9143,1.4402,1.7296,1.0371,6.8258,5.1668,7.7564,46.13,91.48,3.385
2,2,2010-01-06,0.9189,1.4404,1.7292,1.0333,6.8272,5.1638,7.7546,45.72,92.53,3.379
3,3,2010-01-07,0.9168,1.4314,1.7409,1.0351,6.8280,5.1981,7.7539,45.67,93.31,3.368
4,4,2010-01-08,0.9218,1.4357,1.7342,1.0345,6.8274,5.1827,7.7553,45.50,92.70,3.375


In [3]:
def preprocess(data):
    data.drop("Unnamed: 0.1", axis=1, inplace=True)
    data.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
    data["Date"] = pd.to_datetime(data["Date"])
    data.set_index("Date", inplace=True)
    data.replace(0, np.nan, inplace=True)
    display(data)
    print("Filling missing Values: ")
    display(data.interpolate(method="linear", limit_direction="forward"))
    data.interpolate(method="linear", limit_direction="forward", inplace=True)
    return data

In [4]:
data = preprocess(data)

,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Date,,,,,,,,,,
2010-01-04,0.9133,1.4419,1.7200,1.0377,6.8273,5.1597,7.7555,46.27,92.55,3.3960
2010-01-05,0.9143,1.4402,1.7296,1.0371,6.8258,5.1668,7.7564,46.13,91.48,3.3850
2010-01-06,0.9189,1.4404,1.7292,1.0333,6.8272,5.1638,7.7546,45.72,92.53,3.3790
2010-01-07,0.9168,1.4314,1.7409,1.0351,6.8280,5.1981,7.7539,45.67,93.31,3.3680
2010-01-08,0.9218,1.4357,1.7342,1.0345,6.8274,5.1827,7.7553,45.50,92.70,3.3750
...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0.6978,1.1174,4.0507,1.3073,6.9954,6.6829,7.7874,71.45,109.47,4.1260
2019-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Filling missing Values: 


,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Date,,,,,,,,,,
2010-01-04,0.913300,1.441900,1.720000,1.0377,6.8273,5.1597,7.755500,46.27,92.550000,3.3960
2010-01-05,0.914300,1.440200,1.729600,1.0371,6.8258,5.1668,7.756400,46.13,91.480000,3.3850
2010-01-06,0.918900,1.440400,1.729200,1.0333,6.8272,5.1638,7.754600,45.72,92.530000,3.3790
2010-01-07,0.916800,1.431400,1.740900,1.0351,6.8280,5.1981,7.753900,45.67,93.310000,3.3680
2010-01-08,0.921800,1.435700,1.734200,1.0345,6.8274,5.1827,7.755300,45.50,92.700000,3.3750
...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0.697800,1.117400,4.050700,1.3073,6.9954,6.6829,7.787400,71.45,109.470000,4.1260
2019-12-28,0.698667,1.118833,4.038867,1.3068,6.9924,6.6749,7.786833,71.40,109.263333,4.1191
2019-12-29,0.699533,1.120267,4.027033,1.3063,6.9894,6.6669,7.786267,71.35,109.056667,4.1122


In [5]:
LOOK_BACK = 30
PREDICT_DAY = 1
SPLIT_RATIO = 0.8

In [6]:
def Create_Data(
    data, lookback=LOOK_BACK, pred_len=PREDICT_DAY, split_ratio=SPLIT_RATIO, model="FNN"
):
    if lookback < 2:
        print("ERROR: Lookback too small")
        return -1

    # declarations

    x = {}
    y = {}
    xtr = {}
    xt = {}
    ytr = {}
    yt = {}
    scalers = {}

    # Creating stepped data

    for i in data.columns:
        xtemp = pd.DataFrame(data[i])
        for j in range(1, lookback + 1):
            xtemp[i + str(j)] = data[i].shift(-1 * j)
        x[i] = xtemp.dropna()

    # Splitting data into x and y

    for i in x.keys():
        y[i] = pd.DataFrame(x[i].iloc[:, -pred_len])
        x[i] = x[i].iloc[:, :-pred_len]

    # Normalizing x and y values

    for i in x.keys():
        scalers[i + "_x"] = MinMaxScaler(feature_range=(0, 1))
        x[i] = scalers[i + "_x"].fit_transform(x[i])
        scalers[i + "_y"] = MinMaxScaler(feature_range=(0, 1))
        y[i] = scalers[i + "_y"].fit_transform(y[i])

    # setting train and test sizes

    tr_len = int(split_ratio * y["India"].shape[0])
    t_len = y["India"].shape[0] - tr_len

    # creating training and testing data

    for i in x.keys():
        xtr[i] = x[i][:tr_len]
        ytr[i] = y[i][:tr_len]
        xt[i] = x[i][-t_len:]
        yt[i] = y[i][-t_len:]

    # returning pertinent data

    return x, y, xtr, xt, ytr, yt, scalers

In [7]:
x, y, xtr, xt, ytr, yt, scalers = Create_Data(data, model="RNN")

In [8]:
def Create_model(x, y, lookback=LOOK_BACK, Pred_size=PREDICT_DAY):
    models = {}
    for i in x.keys():
        models[i] = Sequential()
        models[i].add(
            SimpleRNN(
                32,
                input_shape=(
                    LOOK_BACK,
                    1,
                ),
                return_sequences=True,
                activation="relu",
            )
        )
        models[i].add(SimpleRNN(64, return_sequences=True, activation="relu"))
        models[i].add(Dropout(0.2))
        models[i].add(SimpleRNN(128, return_sequences=True, activation="relu"))
        models[i].add(Dropout(0.2))
        models[i].add(SimpleRNN(64, return_sequences=True, activation="relu"))
        models[i].add(Dropout(0.2))
        models[i].add(SimpleRNN(16, activation="relu"))
        models[i].add(Dense(Pred_size))
        models[i].compile(loss="mean_squared_error", optimizer="adam")
        print(i)
    return models

In [9]:
m = Create_model(x, y)

Australia
Europe
Brazil
Canada
China
Denmark
Hong Kong
India
Japan
Malaysia


In [10]:
def Execute_model(model, xtr, ytr, xt, yt, scaler):
    MAPE = {}
    MAE = {}
    MSE = {}
    MSE_Actual = {}
    MAE_Actual = {}
    MAPE_Actual = {}
    for i in model.keys():
        print(i)
        # Creating EarlyStopping and ReduceLROnPlateau callbacks
        es = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10)
        reduce_lr = ReduceLROnPlateau(
            monitor="val_loss", factor=0.2, patience=5, min_lr=0.0001
        )

        # Training the model with EarlyStopping and ReduceLROnPlateau callbacks
        model[i].fit(
            xtr[i],
            ytr[i],
            epochs=100,
            batch_size=64,
            verbose=1,
            validation_split=0.2,
            callbacks=[es, reduce_lr],
        )

        # collecting predicted and actual values
        temp = model[i].predict(xt[i])
        pred = scaler[i + "_y"].inverse_transform(temp)
        act = scaler[i + "_y"].inverse_transform(yt[i])

        # calculating Mean Square Error, Mean Absolute Error, and Mean Absolute Error
        MSE[i] = mean_squared_error(yt[i], temp)
        MAE[i] = mean_absolute_error(yt[i], temp)
        MAPE[i] = mean_absolute_percentage_error(yt[i], temp)
        
        MSE_Actual[i] = mean_squared_error(act, pred)
        MAE_Actual[i] = mean_absolute_error(act, pred)
        MAPE_Actual[i] = mean_absolute_percentage_error(act, pred)
        
    # Tabulating Data
    results = pd.DataFrame([MSE_Actual, MAE_Actual, MAPE_Actual])
    results["Metric"] = ["MSE", "MAE", "MAPE"]
    results.set_index("Metric", inplace=True)
    
    metric = pd.DataFrame([MSE, MAE, MAPE])
    metric["Metric"] = ["MSE", "MAE", "MAPE"]
    metric.set_index("Metric", inplace=True)

    return metric, results

In [11]:
metric, result = Execute_model(m, xtr, ytr, xt, yt, scalers)

Australia
Epoch 1/100
37/37 [==============================] - 3s 26ms/step - loss: 0.0189 - val_loss: 8.6489e-04 - lr: 0.0010
Epoch 2/100
37/37 [==============================] - 1s 19ms/step - loss: 0.0065 - val_loss: 8.5654e-04 - lr: 0.0010
Epoch 3/100
37/37 [==============================] - 1s 19ms/step - loss: 0.0049 - val_loss: 0.0014 - lr: 0.0010
Epoch 4/100
37/37 [==============================] - 1s 19ms/step - loss: 0.0034 - val_loss: 3.0066e-04 - lr: 0.0010
Epoch 5/100
37/37 [==============================] - 1s 20ms/step - loss: 0.0037 - val_loss: 4.1981e-04 - lr: 0.0010
Epoch 6/100
37/37 [==============================] - 1s 20ms/step - loss: 0.0025 - val_loss: 5.4137e-04 - lr: 0.0010
Epoch 7/100
37/37 [==============================] - 1s 20ms/step - loss: 0.0024 - val_loss: 0.0013 - lr: 0.0010
Epoch 8/100
37/37 [==============================] - 1s 20ms/step - loss: 0.0021 - val_loss: 0.0011 - lr: 0.0010
Epoch 9/100
37/37 [==============================] - 1s 20ms/step 

Epoch 5/100
37/37 [==============================] - 1s 38ms/step - loss: 0.0011 - val_loss: 0.0079 - lr: 0.0010
Epoch 6/100
37/37 [==============================] - 1s 38ms/step - loss: 9.8389e-04 - val_loss: 0.0077 - lr: 0.0010
Epoch 7/100
37/37 [==============================] - 1s 38ms/step - loss: 7.9924e-04 - val_loss: 0.0086 - lr: 0.0010
Epoch 8/100
37/37 [==============================] - 1s 38ms/step - loss: 7.5693e-04 - val_loss: 0.0062 - lr: 0.0010
Epoch 9/100
37/37 [==============================] - 1s 38ms/step - loss: 8.0607e-04 - val_loss: 0.0081 - lr: 0.0010
Epoch 10/100
37/37 [==============================] - 1s 38ms/step - loss: 6.0196e-04 - val_loss: 0.0063 - lr: 2.0000e-04
Epoch 11/100
37/37 [==============================] - 1s 38ms/step - loss: 5.8937e-04 - val_loss: 0.0073 - lr: 2.0000e-04
Epoch 12/100
37/37 [==============================] - 1s 38ms/step - loss: 5.7137e-04 - val_loss: 0.0063 - lr: 2.0000e-04
Epoch 13/100
37/37 [==============================] -

23/23 [==============================] - 1s 7ms/step
Malaysia
Epoch 1/100
37/37 [==============================] - 4s 43ms/step - loss: 0.0212 - val_loss: 0.0136 - lr: 0.0010
Epoch 2/100
37/37 [==============================] - 1s 31ms/step - loss: 0.0032 - val_loss: 0.0081 - lr: 0.0010
Epoch 3/100
37/37 [==============================] - 1s 37ms/step - loss: 0.0024 - val_loss: 0.0100 - lr: 0.0010
Epoch 4/100
37/37 [==============================] - 1s 38ms/step - loss: 0.0019 - val_loss: 0.0119 - lr: 0.0010
Epoch 5/100
37/37 [==============================] - 1s 38ms/step - loss: 0.0017 - val_loss: 0.0089 - lr: 0.0010
Epoch 6/100
37/37 [==============================] - 1s 38ms/step - loss: 0.0014 - val_loss: 0.0158 - lr: 0.0010
Epoch 7/100
37/37 [==============================] - 1s 38ms/step - loss: 0.0014 - val_loss: 0.0147 - lr: 0.0010
Epoch 8/100
37/37 [==============================] - 1s 38ms/step - loss: 0.0011 - val_loss: 0.0199 - lr: 2.0000e-04
Epoch 9/100
37/37 [===========

In [12]:
result

,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Metric,,,,,,,,,,
MSE,0.000092,0.000165,0.117053,0.003145,0.009428,0.075336,0.000413,16.223124,27.046450,0.023280
MAE,0.007417,0.009646,0.329529,0.055618,0.086069,0.269194,0.019696,3.975213,5.160363,0.151104
MAPE,0.010347,0.008171,0.085653,0.042320,0.012508,0.041226,0.002512,0.057013,0.046984,0.036873


In [13]:
metric

,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Metric,,,,,,,,,,
MSE,4.887619e-04,0.000816,0.015799,0.011886,0.007275,0.016266,0.040854,0.017520,0.010879,0.009578
MAE,1.713779e-02,0.021436,0.121066,0.108123,0.075605,0.125084,0.195782,0.130635,0.103497,0.096924
MAPE,3.678497e+11,0.074928,0.143460,0.151088,0.109653,0.180786,0.222440,0.154875,0.151664,0.130820


In [14]:
import json
import os

# Define the filename
filename = "./results.json"

# Check if the file exists
if os.path.isfile(filename):
    # If the file exists, load the existing data
    with open(filename, "r") as f:
        data = json.load(f)
else:
    # If the file doesn't exist, create an empty dictionary
    data = {}

# Add the result to the dictionary
data["RNN"] = metric.to_dict()

# Write the dictionary to the file
with open(filename, "w") as f:
    json.dump(data, f)